In [1]:
import numpy as np
from docplex.cp.model import *

In [17]:
def calc_obj(x,proc_t,return_max=True):
    #calc value of objective function
    if return_max:
        return max(np.array(np.transpose(x)).dot(np.array(proc_t)))
    return np.array(np.transpose(x)).dot(np.array(proc_t))

def solve(proc_t,nb_m):
    #params: processing times, number of machines 
    mdl = CpoModel()
   
    #add bin decision variables
    x=[]
    for job_id,proc_job in enumerate(proc_t):
        x_row=[]
        for i in range(nb_m):
            curr_x=mdl.integer_var(0, 1,'x_%i%i' %(job_id,i))
            x_row.append(curr_x)
        x.append(x_row)
    #add every job needs to be mapped to exactly one machine constraint
    for job_id,proc_job in enumerate(proc_t):
        mdl.add(mdl.sum([x[job_id][mach_id] for mach_id in range(nb_m)])==1)
    

    #add lexicographic ordering
    """
    for i in range(nb_m):
        if job_id < nb_m-1:
            mdl.lexicographic(np.array(x)[:,job_id],np.array(x)[:,job_id+1])
    """
        
    #calculate sum of all job durations for every machine
    obj_var=[sum(x[job_id][mach_id]*proc_t[job_id] for job_id,proc_job in enumerate(proc_t)) for mach_id in range(nb_m)]
    
    
    
    
    #minimize the max duration of all machines
    mdl.minimize(mdl.max(obj_var))
    import time
    start = time.time()
    msol=mdl.solve()
    duration = time.time() - start
    sol=[]
    sol_row=[]

    for i,el in enumerate(msol.solution.var_solutions_list):
        sol_row.append(el.value)
        if((i+1)%nb_m==0):
            sol.append(sol_row)
            sol_row=[]
   
        
    return np.array(sol),duration

In [38]:
def simulation(pt_mean,pt_std,nb_t_range,nb_m_range,nb_data,pt_int=False,return_dur=False):
    # init parameters
    # pt_mean: mean of processtime (duration of task on every machine)
    # pt_std:std of processtime 
    # number of task range f.e. [2,10] means uniform distribution between 2 and 10 tasks
    # number of machine range f.e. [2,10] means uniform distribution between 2 and 10 machines
    # nb_data = number of datasamples which should get generated
    # pt_int = (boolean) if process time is int or float (default: False)
    # return_dur = process time get's return (for statistics, default:False)
    x=[]
    y=[]
    if return_dur:
        dur=[]
    for i in range(nb_data):
        
        nb_m=np.random.randint(nb_m_range[0],nb_m_range[1])
        nb_t=np.random.randint(nb_t_range[0],nb_t_range[1])
        print(i,nb_m,nb_t)
        if pt_int:
            proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=nb_t)
            proc_t=proc_t.astype(np.int64)
        else:
            proc_t=np.random.normal(loc=pt_mean,scale=pt_std,size=nb_t)
        x.append(proc_t)
        sol=solve(proc_t,nb_m)
        y.append(sol)
        if return_dur:
            dur.append(sol[-1])
    if return_dur:
        return x,y,dur
    return x,y

x,y=simulation(30,7,[3,17],[2,5],1000,True)
    

0 3 14
1 3 15
2 4 8
3 4 8
4 3 15
5 3 3
6 3 3
7 4 12
8 2 8
9 3 3
10 3 9
11 4 12
12 3 12
13 3 10
14 3 12
15 2 8
16 3 15
17 3 13
18 3 6
19 2 13
20 2 5
21 3 16
22 2 15
23 4 11
24 4 5
25 4 16
26 4 8
27 3 14
28 2 9
29 2 4
30 2 15
31 2 12
32 2 10
33 4 15
34 2 14
35 3 13
36 2 10
37 3 13
38 3 8
39 4 8
40 2 14
41 4 16
42 2 10
43 3 16
44 3 9
45 4 7
46 2 8
47 4 8
48 2 15
49 3 9
50 2 11
51 3 3
52 3 6
53 4 14
54 2 4
55 2 15
56 3 14
57 3 12
58 2 10
59 2 3
60 3 4
61 2 3
62 4 11
63 4 13
64 2 4
65 2 12
66 3 14
67 2 8
68 3 14
69 4 15
70 3 12
71 4 15
72 2 6
73 4 12
74 3 4
75 3 11
76 2 8
77 3 8
78 4 13
79 2 12
80 3 6
81 4 5
82 2 11
83 4 4
84 2 11
85 2 9
86 4 12
87 3 4
88 4 11
89 2 8
90 2 10
91 3 7
92 4 14
93 3 12
94 3 12
95 2 14
96 2 16
97 3 8
98 3 5
99 3 8
100 4 14
101 2 12
102 3 12
103 2 8
104 2 3
105 2 6
106 3 15
107 2 6
108 4 11
109 2 6
110 2 5
111 2 5
112 3 13
113 3 15
114 4 14
115 3 5
116 4 12
117 2 5
118 2 10
119 3 7
120 2 9
121 3 8
122 4 4
123 2 3
124 3 15
125 3 10
126 3 15
127 4 14
128 2 6
129 3 9

981 3 9
982 3 6
983 4 14
984 2 6
985 3 9
986 2 14
987 2 12
988 2 15
989 3 5
990 3 9
991 4 8
992 2 9
993 2 15
994 4 6
995 4 11
996 4 16
997 3 15
998 4 15
999 3 4


In [39]:
x


[array([22, 43, 27, 23, 25, 27, 28, 44, 32, 27, 31, 33, 41, 25],
       dtype=int64),
 array([23, 31, 30, 31, 23, 26, 32, 32, 26, 32, 31, 21, 37, 45, 30],
       dtype=int64),
 array([23, 11, 28, 19, 36, 33, 35, 27], dtype=int64),
 array([27, 25, 18, 29, 30, 19, 38, 35], dtype=int64),
 array([36, 36, 22, 31, 32, 30, 20, 33, 37, 34, 32, 38, 29, 22, 13],
       dtype=int64),
 array([35, 39, 26], dtype=int64),
 array([34, 29, 30], dtype=int64),
 array([26, 24, 27, 32, 26, 30, 31, 33, 37, 19, 26, 28], dtype=int64),
 array([37, 40, 33, 27, 31, 41, 26, 25], dtype=int64),
 array([28, 38, 30], dtype=int64),
 array([30, 28, 16, 24, 28, 32, 23, 38, 27], dtype=int64),
 array([38, 30, 27, 20, 34, 24, 33, 29, 34, 12, 36, 32], dtype=int64),
 array([38, 29, 30, 35, 28, 28, 29, 21, 30, 13, 37, 24], dtype=int64),
 array([25, 23, 42, 28, 26, 33, 39, 38, 25, 27], dtype=int64),
 array([20, 31, 30, 38, 33, 24, 28, 29, 31, 35, 28, 30], dtype=int64),
 array([33, 28, 28, 25, 27, 30, 51, 44], dtype=int64),
 ar

In [40]:
y

[(array([[0, 0, 1],
         [0, 0, 1],
         [0, 0, 1],
         [0, 0, 1],
         [0, 1, 0],
         [0, 1, 0],
         [0, 1, 0],
         [1, 0, 0],
         [0, 1, 0],
         [0, 0, 1],
         [0, 1, 0],
         [1, 0, 0],
         [1, 0, 0],
         [1, 0, 0]]),
  0.9582171440124512),
 (array([[0, 0, 1],
         [0, 0, 1],
         [1, 0, 0],
         [0, 1, 0],
         [0, 1, 0],
         [1, 0, 0],
         [0, 0, 1],
         [0, 0, 1],
         [1, 0, 0],
         [0, 0, 1],
         [1, 0, 0],
         [0, 1, 0],
         [1, 0, 0],
         [0, 1, 0],
         [0, 1, 0]]),
  2.742635488510132),
 (array([[0, 0, 1, 0],
         [0, 0, 0, 1],
         [1, 0, 0, 0],
         [0, 1, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0]]),
  0.03800773620605469),
 (array([[0, 0, 0, 1],
         [0, 0, 1, 0],
         [1, 0, 0, 0],
         [0, 0, 0, 1],
         [0, 0, 1, 0],
         [0, 1, 0, 0],
         [1, 0, 0, 0],


In [85]:
#write input to .csv
with open('input_pt.csv', 'w') as f:
   
       
    for row in x:
        line=""
        for i,pl in enumerate(row):
            if i!=len(row)-1:
                line=line+str(pl)+','
        f.write(line+str(pl)+'\n')
    str_out      

In [98]:
#write output to .csv
with open('output.csv', 'w') as of:
       
    for row in np.array(y)[:,0]:
        line="["
        for i,elem in enumerate( row):
            line=line+'['
            for j,nb in enumerate( elem):
                if j!=len(elem)-1:

                    line=line+str(nb)+','
                
            
            if i!=len(row)-1:
                line=line+str(nb)+'],'
            
        of.write(line+str(nb)+']]\n')       
        